In [1]:
%pip install azure-cognitiveservices-speech
%pip install azure-ai-textanalytics
# %pip install azure-core
%pip install --upgrade gradio
%pip install gTTS

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import gradio as gr
import azure.cognitiveservices.speech as speechsdk
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
import re
from gtts import gTTS

import warnings
warnings.filterwarnings("ignore")

/opt/anaconda3/envs/BIAL/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
azure_text_analysis_key = "<Get the key from Azure>"
azure_speech_service_key = "<Get the key from Azure>"

In [4]:
# Method to expand and return abbrevations
def expand_abbreviations(text):
    # Dictionary of abbreviations and their expansions
    abbreviations = {
        "INR": "Indian Rupees",
        "USD": "US Dollars",
        "ETA": "Estimated Time of Arrival",
        "ETD": "Estimated Time of Departure",
        "<DE>": "<lang xml:lang='de-DE'>",
        "</DE>": "</lang"
    }

    # Regex pattern for all keys in the dictionary
    pattern = r'\b(' + '|'.join(re.escape(key) for key in abbreviations.keys()) + r')\b'
    expanded_text = re.sub(pattern, lambda match: abbreviations[match.group(0)], text)

    return expanded_text

In [5]:
# SSML generating function based on various arguments, but primarily selection is on sentiment
def generate_ssml_based_on_sentiment(sentiment, text, voice, style, styledegree, pitch, rate):
    # Expand abbreviations in the text (assuming you have an expand_abbreviations function)
    text = expand_abbreviations(text)

    # Block showcasing handling of Dutch translation using multi-lingual voice agent
    if "<DE>" in text:
        voice = "en-US-AvaMultilingualNeural"
        return f"""
        <speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis" xmlns:mstts="https://www.w3.org/2001/mstts" xml:lang="de-DE">
            <voice name="{voice}">
                <mstts:express-as style="{style}" styledegree="{styledegree}">
                    <prosody pitch="{pitch}" rate="{rate}">
                        {text}
                    </prosody>
                </mstts:express-as>
            </voice>
        </speak>
        """
    
    if sentiment == 'positive':
        ssml = f"""
        <speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis" xmlns:mstts="https://www.w3.org/2001/mstts" xml:lang="en-IN">
            <voice name="{voice}">
                <mstts:express-as style="{style}" styledegree="{styledegree}">
                    <prosody pitch="{pitch}" rate="{rate}">
                        {text}
                    </prosody>
                </mstts:express-as>
            </voice>
        </speak>
        """
    elif sentiment == 'neutral':
        ssml = f"""
        <speak version='1.0' xmlns='http://www.w3.org/2001/10/synthesis' xmlns:mstts="https://www.w3.org/2001/mstts" xml:lang='en-IN'>
            <voice name='{voice}'>
                <prosody pitch="{pitch}" rate="{rate}">
                    {text}
                </prosody>
            </voice>
        </speak>
        """
    else:  # negative sentiment
        ssml = f"""
        <speak version='1.0' xmlns='http://www.w3.org/2001/10/synthesis' xmlns:mstts="https://www.w3.org/2001/mstts" xml:lang='en-IN'>
            <voice name='{voice}'>
                <mstts:express-as style="{style}" styledegree="{styledegree}"> 
                    <prosody pitch="{pitch}" rate="{rate}">
                        {text}
                    </prosody>
                </mstts:express-as>
            </voice>
        </speak>
        """
    return ssml

In [6]:
# Azure Text Analytics setup
def authenticate_client():
    key = azure_text_analysis_key
    endpoint = "https://aknaraya-text-analysois.openai.azure.com/"
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(endpoint=endpoint, credential=ta_credential)
    return text_analytics_client

def analyze_sentiment(client, text):
    response = client.analyze_sentiment([text])[0]
    return response.sentiment  # Returns 'positive', 'neutral', or 'negative'

In [7]:
def synthesize_speech_based_on_sentiment(ssml):
    # Azure Speech Config
    speech_config = speechsdk.SpeechConfig(subscription=azure_speech_service_key, region="centralindia")
    
    # Synthesizer using SSML
    synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config)
    result = synthesizer.speak_ssml_async(ssml).get()

    # Check synthesis result
    if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print("Speech synthesized successfully.")
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        print(f"Speech synthesis canceled: {cancellation_details.reason}")
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print(f"Error details: {cancellation_details.error_details}")


In [8]:
sentences_by_sentiment = [
        "<DE>Wir freuen uns auf die Zusammenarbeit mit Ihnen!</DE>",
        "Welcome to the airport! I hope you’re having a great day. How can I assist you with your travel plans today?",
        "If you’d like to do some last-minute shopping, there are duty-free stores available, and they accept payments in INR, USD, or via popular payment apps.",
        "I’d like to inform you that your flight to Delhi, Indigo 6E 456, has been delayed by an hour. The new departure time is 8:45 PM on October 22nd. In the meantime, you might want to check out Haldiram’s near the food court.",
        "I’m delighted to help you! If you follow me, I can guide you directly to Gate 21, which is just a short walk from here.",
        "Thank you so much for your patience during this busy time; I genuinely appreciate your understanding.",
        "Rest assured, you’re in capable hands, and we’ll do everything we can to ensure you catch your flight to New York on October 22nd at 3:15 PM without any issues!",
        "I have some fantastic news! Your flight, Delta flight 123, is running on schedule, so feel free to enjoy the lounge until it's time to board.",
        "The check-in process is quite straightforward, so if you follow the signs, you’ll find it easy to navigate through.",
        "Your gate is conveniently located at the end of this terminal, just a short walk from here.",
        "At this moment, we are operating under normal circumstances, and there are no major disruptions to report.",
        "Currently, the estimated wait time for security clearance is approximately 15 minutes, so you should be fine.",
        "Boarding for your flight will commence shortly, so please keep an ear out for any announcements regarding your flight.",
        "I’m truly sorry to inform you that your flight has unfortunately been delayed until 4:30 PM on October 22nd due to unforeseen circumstances.",
        "Regrettably, we cannot accommodate any additional baggage on that flight as we have reached our maximum capacity.",
        "I completely understand your concern, but unfortunately, we are unable to make any exceptions to the ticket rules at this time.",
        "I sincerely apologize for the extended wait; we are currently experiencing an unusually high volume of passengers today.",
        "I regret to inform you that we do not have any record of your reservation under the name John Smith in our system, which is very concerning."
]

In [9]:
# Options for dropdown menus
voices = ['en-IN-NeerjaNeural', 'en-IN-PrabhatNeural']  # Add other available voices
styles = ['default', 'cheerful', 'empathetic', 'newscast']
style_degrees = ['1', '0.01', '2']  # Assuming these degrees
pitches = ['default', 'low', 'high']
rates = ['default', 'slow', 'fast']

In [10]:
client = authenticate_client()

In [11]:
# Dropdown choices from the array
dropdown_choices = [(f"Sentence {i+1}: {sentence}", sentence) for i, sentence in enumerate(sentences_by_sentiment)]

# Process user choices, detect sentiment and process speech
def process_input(selected_sentence, voice, style, style_degree, pitch, rate):
    sentiment = analyze_sentiment(client, selected_sentence)  # Analyze sentiment of the selected sentence
    ssml = generate_ssml_based_on_sentiment(sentiment, selected_sentence, voice, style, style_degree, pitch, rate)
    response = synthesize_speech_based_on_sentiment(ssml)
    return f"Detected sentiment: {sentiment}."


In [13]:
# Gradio UI setup with improved layout and styling
with gr.Blocks(css=".header {text-align: center; font-size: 24px; margin: 10px;}") as iface:
    # Header
    gr.Markdown("<h1 class='header'>SSML Text-to-Speech with Sentiment Analysis</h1>")
    
    with gr.Row():  # Row layout for sidebar and output
        # Sidebar for inputs
        with gr.Column(scale=1):  # Smaller column for inputs
            gr.Markdown("### Select Options")  # Section header for inputs
            voice_dropdown = gr.Dropdown(choices=voices, label="Select Voice")  # Voice dropdown
            style_dropdown = gr.Dropdown(choices=styles, label="Select Speech Style")  # Speech style dropdown
            style_degree_dropdown = gr.Dropdown(choices=style_degrees, label="Select Style Degree")  # Style degree dropdown
            pitch_dropdown = gr.Dropdown(choices=pitches, label="Select Pitch")  # Pitch dropdown
            rate_dropdown = gr.Dropdown(choices=rates, label="Select Rate")  # Rate dropdown
            submit_button = gr.Button("Generate Speech", variant="primary")  # Primary button to submit and generate speech
        
        # Main content/output section
        with gr.Column(scale=2):  # Larger column for output
            gr.Markdown("### Input Sentence")  # Section header for user input
            input_textbox = gr.Textbox(
                label="User Input", 
                placeholder="Type your sentence here...", 
                lines=2  # Slightly larger for typing
            )
            gr.Markdown("### Generated Output")  # Section header for output
            output_text = gr.Textbox(
                label="Output", 
                placeholder="The synthesized speech output will appear here...", 
                lines=5,
                interactive=False  # Non-editable textbox
            )

    # Link the button to the function that processes the input
    submit_button.click(
        process_input, 
        inputs=[input_textbox, voice_dropdown, style_dropdown, style_degree_dropdown, pitch_dropdown, rate_dropdown], 
        outputs=output_text
    )

# Launch the Gradio interface
iface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Speech synthesized successfully.
Info: on_underlying_io_bytes_received: Close frame received
Info: on_underlying_io_bytes_received: closing underlying io.
Info: on_underlying_io_close_complete: uws_state: 6.
